In [ ]:
%pip install langdetect
%pip install langid
%pip install fasttext
%pip install pycld3
%pip install -U git+https://github.com/aboSamoor/polyglot.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-40m_bbs4
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /tmp/pip-req-build-40m_bbs4


In [ ]:
import pandas as pd
import numpy as np
from langdetect import DetectorFactory, detect_langs
import langid
import fasttext
import cld3
from polyglot.text import Text, Word

In [ ]:
df = pd.read_csv('posts/language_labels.tsv', delimiter = '\t')
print(len(df))
df.head()

450


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,message,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,label,Unnamed: 11
0,NaN,NaN,NaN,1,За роки війни я звик на волонтерство витрачати...,NaN,NaN,NaN,NaN,NaN,u,NaN
1,NaN,NaN,NaN,2,"⚡️ В Іспанії затримали Анатолія Шарія, якого п...",NaN,NaN,NaN,NaN,NaN,u,NaN
2,NaN,NaN,NaN,3,ОПЕРАТИВНА ІНФОРМАЦІЯ ЩОДО РАДІАЦІЙНОГО ФОНУ н...,NaN,NaN,NaN,NaN,NaN,u,NaN
3,NaN,NaN,NaN,4,У Луганській області окупанти обстріляли еваку...,NaN,NaN,NaN,NaN,NaN,u,NaN
4,NaN,NaN,NaN,5,NOVÝ ŠÉF BRITSKÉHO GENERÁLNEHO ŠTÁBU CHCE BUDO...,NaN,NaN,NaN,NaN,NaN,xx,NaN


In [ ]:
# drop unnamed colums 
for column in df.columns:
  if column.startswith('Unnamed'):
    df.drop([column], axis=1, inplace=True)

df.head()

,message,label
0,За роки війни я звик на волонтерство витрачати...,u
1,"⚡️ В Іспанії затримали Анатолія Шарія, якого п...",u
2,ОПЕРАТИВНА ІНФОРМАЦІЯ ЩОДО РАДІАЦІЙНОГО ФОНУ н...,u
3,У Луганській області окупанти обстріляли еваку...,u
4,NOVÝ ŠÉF BRITSKÉHO GENERÁLNEHO ŠTÁBU CHCE BUDO...,xx


In [ ]:
# langdetect
DetectorFactory.seed = 0
def set_label_langdetect(message: str) -> str:
  try:
    label = detect_langs(message)
    if len(label) == 1:
      if label[0].lang == "uk":
        return 'u'
      elif label[0].lang == "ru":
        return 'r'
      else:
        return 'xx'
    elif len(label) >= 2:
      if label[0].lang == "uk":
        if label[0].prob > 0.9:
          return 'u'
        elif label[0].prob > 0.3 and label[1].lang == "ru" and label[1].prob > 0.3:
          return 'mix'
        else:
          return 'uplus'
      elif label[0].lang == "ru":
        if label[0].prob > 0.9:
          return 'r'
        elif label[0].prob > 0.3 and label[1].lang == "uk" and label[1].prob > 0.3:
          return 'mix'
        else:
          return 'rplus'
      elif label[0].prob > 0.3 and label[1].lang == "uk" and label[1].prob > 0.3:
          return 'uplus'
      elif label[0].prob > 0.3 and label[1].lang == "uk" and label[1].prob > 0.3:
          return 'rplus'
      else:
        return 'xx'
  except:
    return 'xx'

In [ ]:
# langid
def set_label_langid(message: str) -> str:
  label = langid.classify(message)
  if label[0] == 'uk':
    return 'u'
  elif label[0] == 'ru':
    return 'r'
  else:
    return 'xx'

In [ ]:
# fasttext
# donload model here https://fasttext.cc/docs/en/language-identification.html
model = fasttext.load_model('lid.176.ftz')
def set_label_fasttext(message: str) -> str:
  label = model.predict(message, k=2)
  if label[0][0] == '__label__uk':
    if label[1][0] > 0.9:
      return 'u'
    elif label[1][0] > 0.3 and label[0][1] == '__label__ru' and label[1][1] > 0.3:
      return 'mix'
    else:
      return 'uplus'
  elif label[0][0] == '__label__ru':
    if label[1][0] > 0.9:
      return 'r'
    elif label[1][0] > 0.3 and label[0][1] == '__label__uk' and label[1][1] > 0.3:
      return 'mix'
    else:
      return 'rplus'
  elif label[1][0] > 0.3 and label[0][1] == '__label__uk' and label[1][1] > 0.3:
    return 'uplus'
  elif label[1][0] > 0.3 and label[0][1] == '__label__ru' and label[1][1] > 0.3:
    return 'rplus'
  else:
    return 'xx'

In [ ]:
# cld3
def set_label_cld3(message: str) -> str:
  label = cld3.get_language(message)
  if label[0] == 'uk':
    return 'u'
  elif label[0] == 'ru':
    return 'r'
  else:
    return 'xx'

In [ ]:
# polyglot
def set_label_polyglot(message: str) -> str:
  text = Text(message)
  label = text.language.code
  if label == 'uk':
    return 'u'
  elif label == 'ru':
    return 'r'
  else:
    return 'xx'

In [ ]:
df['langdetect'] =  np.vectorize(set_label_langdetect)(df['message'])
df['langid'] =  np.vectorize(set_label_langid)(df['message'])
df['fasttext'] =  np.vectorize(set_label_fasttext)(df['message'])
df['cld3'] =  np.vectorize(set_label_cld3)(df['message'])
df['polyglot'] =  np.vectorize(set_label_polyglot)(df['message'])
df

,message,label,langdetect,langid,fasttext,cld3,polyglot
0,За роки війни я звик на волонтерство витрачати...,u,u,u,u,u,u
1,"⚡️ В Іспанії затримали Анатолія Шарія, якого п...",u,u,u,u,u,u
2,ОПЕРАТИВНА ІНФОРМАЦІЯ ЩОДО РАДІАЦІЙНОГО ФОНУ н...,u,u,u,u,u,u
3,У Луганській області окупанти обстріляли еваку...,u,u,u,u,u,u
4,NOVÝ ŠÉF BRITSKÉHO GENERÁLNEHO ŠTÁBU CHCE BUDO...,xx,xx,xx,xx,xx,xx
...,...,...,...,...,...,...,...
445,Жодних компромісів щодо територіальної цілісно...,u,u,u,u,u,u
446,Чергова атака російських тварин на столицю віл...,u,u,u,u,u,u
447,Главная ошибка нашей власти и обслуги зачастую...,r,r,r,r,r,r
448,🇺🇦🇪🇺Заступниця Голови Комітет з питань економі...,u,u,u,u,u,u


In [ ]:
df['langdetect_res'] = np.where(df['label'] == df['langdetect'], 1, 0)
df['langid_res'] = np.where(df['label'] == df['langid'], 1, 0)
df['fasttext_res'] = np.where(df['label'] == df['fasttext'], 1, 0)
df['cld3_res'] = np.where(df['label'] == df['cld3'], 1, 0)
df['polyglot_res'] = np.where(df['label'] == df['polyglot'], 1, 0)
df.to_csv('results.csv')
df

,message,label,langdetect,langid,fasttext,cld3,polyglot,langdetect_res,langid_res,fasttext_res,cld3_res,polyglot_res
0,За роки війни я звик на волонтерство витрачати...,u,u,u,u,u,u,1,1,1,1,1
1,"⚡️ В Іспанії затримали Анатолія Шарія, якого п...",u,u,u,u,u,u,1,1,1,1,1
2,ОПЕРАТИВНА ІНФОРМАЦІЯ ЩОДО РАДІАЦІЙНОГО ФОНУ н...,u,u,u,u,u,u,1,1,1,1,1
3,У Луганській області окупанти обстріляли еваку...,u,u,u,u,u,u,1,1,1,1,1
4,NOVÝ ŠÉF BRITSKÉHO GENERÁLNEHO ŠTÁBU CHCE BUDO...,xx,xx,xx,xx,xx,xx,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
445,Жодних компромісів щодо територіальної цілісно...,u,u,u,u,u,u,1,1,1,1,1
446,Чергова атака російських тварин на столицю віл...,u,u,u,u,u,u,1,1,1,1,1
447,Главная ошибка нашей власти и обслуги зачастую...,r,r,r,r,r,r,1,1,1,1,1
448,🇺🇦🇪🇺Заступниця Голови Комітет з питань економі...,u,u,u,u,u,u,1,1,1,1,1


In [ ]:
# calculate accuracy of different libraries
N = len(df)
result = {'langdetect': df['langdetect_res'].sum()/N, 'langid': df['langid_res'].sum()/N,
          'fasttext': df['fasttext_res'].sum()/N, 'cld3': df['cld3_res'].sum()/N,
          'polyglot': df['polyglot_res'].sum()/N}
result

{'cld3': 0.9822222222222222,
 'fasttext': 0.96,
 'langdetect': 0.9644444444444444,
 'langid': 0.9644444444444444,
 'polyglot': 0.9755555555555555}

In [ ]:
def performance_metrics(y_actual, y_hat):
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  for i in range(len(y_hat)): 
      if y_actual[i]==y_hat[i]==1:
          TP += 1
      if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
          FP += 1
      if y_actual[i]==y_hat[i]==0:
          TN += 1
      if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
          FN += 1

  sensitivity  = TP / (TP+FN)
  specificity  = TN / (TN+FP) 
  pos_pred_val = TP / (TP+FP)
  neg_pred_val = TN / (TN+FN)

  return sensitivity, specificity, pos_pred_val, neg_pred_val
  # return TP, TN, FP, FN

In [ ]:
ukr_true = np.where(df['label'] == 'u', 1, 0)
rus_true = np.where(df['label'] == 'r', 1, 0)

ukr_true

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
evaluation_results = pd.DataFrame(columns = ['accuracy', 'sensitivity_ukr', 'specificity_ukr',
                                 'pos_pred_val_ukr', 'neg_pred_val_ukr',
                                 'sensitivity_rus', 'specificity_rus',
                                 'pos_pred_val_rus', 'neg_pred_val_rus'],
                                  index=['langdetect', 'langid', 'fasttext','cld3', 'polyglot'])
evaluation_results

,accuracy,sensitivity_ukr,specificity_ukr,pos_pred_val_ukr,neg_pred_val_ukr,sensitivity_rus,specificity_rus,pos_pred_val_rus,neg_pred_val_rus
langdetect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
langid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fasttext,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cld3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
polyglot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
langdetect_ukr = np.where(df['langdetect'] == 'u', 1, 0)
langdetect_rus = np.where(df['langdetect'] == 'r', 1, 0)
ukr_metrics_langdetect = performance_metrics(ukr_true, langdetect_ukr)
rus_metrics_langdetect = performance_metrics(rus_true, langdetect_rus)
evaluation_results.loc['langdetect'] = [result['langdetect'], *ukr_metrics_langdetect, *rus_metrics_langdetect]

langid_ukr = np.where(df['langid'] == 'u', 1, 0)
langid_rus = np.where(df['langid'] == 'r', 1, 0)
ukr_metrics_langid = performance_metrics(ukr_true, langid_ukr)
rus_metrics_langid = performance_metrics(rus_true, langid_rus)
evaluation_results.loc['langid'] = [result['langid'], *ukr_metrics_langid, *rus_metrics_langid]

fasttext_ukr = np.where(df['fasttext'] == 'u', 1, 0)
fasttext_rus = np.where(df['fasttext'] == 'r', 1, 0)
ukr_metrics_fasttext = performance_metrics(ukr_true, fasttext_ukr)
rus_metrics_fasttext = performance_metrics(rus_true, fasttext_rus)
evaluation_results.loc['fasttext'] = [result['fasttext'], *ukr_metrics_fasttext, *rus_metrics_fasttext]

cld3_ukr = np.where(df['cld3'] == 'u', 1, 0)
cld3_rus = np.where(df['cld3'] == 'r', 1, 0)
ukr_metrics_cld3 = performance_metrics(ukr_true, cld3_ukr)
rus_metrics_cld3 = performance_metrics(rus_true, cld3_rus)
evaluation_results.loc['cld3'] = [result['cld3'], *ukr_metrics_cld3, *rus_metrics_cld3]

polyglot_ukr = np.where(df['polyglot'] == 'u', 1, 0)
polyglot_rus = np.where(df['polyglot'] == 'r', 1, 0)
ukr_metrics_polyglot = performance_metrics(ukr_true, polyglot_ukr)
rus_metrics_polyglot = performance_metrics(rus_true, polyglot_rus)
evaluation_results.loc['polyglot'] = [result['polyglot'], *ukr_metrics_polyglot, *rus_metrics_polyglot]

evaluation_results

,accuracy,sensitivity_ukr,specificity_ukr,pos_pred_val_ukr,neg_pred_val_ukr,sensitivity_rus,specificity_rus,pos_pred_val_rus,neg_pred_val_rus
langdetect,0.964444,0.967347,0.990244,0.991632,0.962085,0.984127,0.987654,0.96875,0.993789
langid,0.964444,0.971429,0.97561,0.979424,0.966184,0.97619,0.981481,0.953488,0.990654
fasttext,0.96,0.971429,0.97561,0.979424,0.966184,0.97619,1.0,1.0,0.990826
cld3,0.982222,0.991837,0.985366,0.987805,0.990196,0.992063,0.996914,0.992063,0.996914
polyglot,0.975556,0.987755,0.97561,0.979757,0.985222,0.97619,0.996914,0.991935,0.990798
